# Regression

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost
import math
#from __future__ import division
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn import  tree, linear_model
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn import ensemble
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plot
from sklearn.metrics import mean_squared_error, r2_score
import datetime as dt
import pickle

In [2]:
df=pd.read_csv('C:/Users/v-savrav/Desktop/30/InvociePrediction30days_02Oct2019_v1.csv')

In [3]:
#merge_df = merge_df.dropna(subset=['days_to_approval'],axis=0) #how='any',merge_df = merge_df.dropna(subset=['received_days'],axis=0) #how='any',
df.drop(['ReceiptDate', 'FirstPOCAssignmentDate', 'LastBusinessUserDate',
        'Doc. Date',
       'Clrng doc.', 'Pmnt date', 'Reference',
       'Clearing', 'Pstng Date', 'CoCd', 'Net due dt', 'Year',
       'Year/month', 'Period', 
       'Bline Date', 'Reference Key', 'Entry Date', 'SubmitDate'
], axis=1, inplace=True)

In [4]:
df.columns

Index(['Country', 'Region', 'Frequency', 'CC', 'received_days',
       'processing_days', 'days_to_approval', 'days_to_be_cleared', 'PaidDays',
       'NewPaymentTerm', 'DocumentNo', 'Type', 'Amount in doc. curr.', 'Curr.',
       'Vendor', 'Discount base amount', 'Discount amount'],
      dtype='object')

In [5]:
X=df[[ 'CC','Vendor',
       'NewPaymentTerm', 'Amount in doc. curr.','Curr.','Type',  
            'Discount amount','DocumentNo', 'Discount base amount','Country',
       'Region', 'Frequency','received_days','processing_days'
       ]]

# Preprocessing

In [6]:
from sklearn import  preprocessing # used for label encoding and imputing NaNs

# One-hot encode the data using pandas get_dummies
X.fillna(0,inplace=True)
for c in X.columns:
    if X[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X[c].values)) 
        X[c] = lbl.transform(list(X[c].values))

C:\Users\v-savrav\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
C:\Users\v-savrav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [7]:
df['Stage']='Stage3'

In [8]:
X_train, X_test = train_test_split(X, test_size = 0.25, random_state = 42)

In [9]:
len(X_test)

56245

# Prediction

In [10]:
# Split the data into training and testing sets
def model_prediction(X_train, X_test,target, model_name):
    
    y_train, y_test = train_test_split(target, test_size = 0.25, random_state = 42)
    xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                               colsample_bytree=1, max_depth=7)
    params = {'n_estimators': 500, 'max_depth': 6, 'min_samples_split': 4,
          'learning_rate': 0.05, 'loss': 'ls'}
    clf = ensemble.GradientBoostingRegressor(**params)
    xgb.fit(X_train,y_train)
    clf.fit(X_train,y_train)
    filename = 'C:/Users/v-savrav/Desktop/30/models/' + model_name+'_model.sav'
    pickle.dump(clf, open(filename, 'wb'))
    filename1 = 'C:/Users/v-savrav/Desktop/30/models/' + model_name+'_model1.sav'
    pickle.dump(xgb, open(filename1, 'wb'))

    pred1 = xgb.predict(X_test)
    pred2 = clf.predict(X_test)
    finalpred =(pred1+pred2)/2
    mae = mean_absolute_error(finalpred ,y_test)
    return finalpred,y_test,mae 


In [11]:
len(X_test)

56245

# Approval

In [12]:
target = df['days_to_approval']
approval_days_prediction,actual_approval,mae= model_prediction(X_train, X_test,target,'stage3_approve')


C:\Users\v-savrav\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[10:10:03] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [13]:
mae

0.12946744554913772

In [14]:
#approvedf=pd.DataFrame({'Actual approval days':actual_approval, 'Predicted Approval days':approval_days_prediction})

approvedf=pd.DataFrame({'Approval - Act':actual_approval, 'Approval - Pr':approval_days_prediction})


# Paid

In [15]:
#df['var_paymenent_days'] = df['PaidDays'] - df['NewPaymentTerm'] 
target=df['PaidDays']
paid_days_prediction,actual_paid,mae2= model_prediction(X_train,X_test,target,'stage3_paid')

C:\Users\v-savrav\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[10:18:57] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [23]:
mae2

1.826517096173878

In [20]:
#paiddf = pd.DataFrame({'Actual Paid days':y_test, 'Predicted  Paid days':paid_days_prediction})

paiddf = pd.DataFrame({'Paid - Act':actual_paid, 'Paid - Pr':paid_days_prediction})


In [ ]:
X_test.to_excel('C:/Users/v-savrav/Desktop/30/Data_test/Stage3.xlsx')

In [22]:
frames=[approvedf,paiddf]
result = pd.concat(frames,axis=1)
final=pd.concat([result, X_test],axis=1)
#final=pd.concat([result, df],axis=1)
#final['Stage']='Stage3'


In [24]:
final

,Approval - Act,Approval - Pr,Paid - Act,Paid - Pr,CC,Vendor,NewPaymentTerm,Amount in doc. curr.,Curr.,Type,Discount amount,DocumentNo,Discount base amount,Country,Region,Frequency,received_days,processing_days
23216,4,4.034157,28,27.300625,1075,3009611,30.0,-1.289880e+03,8,3,0.00,3700002779,-1.209980e+03,12,6,1,3,4
207519,0,0.002327,26,24.286633,1962,2252281,30.0,-8.045352e+04,25,0,0.00,5100003468,-8.045352e+04,37,0,1,15,0
219777,7,6.935653,29,27.488770,1290,3009165,30.0,-3.745000e+01,51,3,0.00,3700024336,-3.500000e+01,72,0,1,37,7
213486,30,30.124110,33,32.008755,1705,3013865,30.0,-3.446165e+05,18,2,0.00,5100018759,-3.446165e+05,56,2,1,4,30
89316,4,4.005709,29,27.046460,1033,2328646,30.0,-2.736402e+06,11,2,0.00,5100111106,-2.820300e+06,15,3,1,0,4
207085,13,12.979355,28,27.765621,1290,2284202,30.0,-1.305650e+03,54,2,0.00,5100293883,-1.305650e+03,72,0,1,-1,13
134447,11,11.037517,22,21.124863,1035,2253661,30.0,-6.415200e+04,59,2,0.00,5100154850,-6.415200e+04,76,4,12,0,11
75778,2,1.947837,29,24.638003,1098,2248347,30.0,-7.975250e+04,25,0,0.00,5100240444,-7.975250e+04,37,0,1,35,2
83353,7,6.937666,26,27.099322,1291,3009165,30.0,-1.551000e+02,51,3,0.00,3700006056,-1.449800e+02,72,0,1,40,7
19968,3,2.985093,26,27.245279,1088,2108583,30.0,-3.132000e+04,37,2,0.00,5100153074,-2.700000e+04,52,3,1,0,3


In [27]:
final.to_excel('C:\\Users\\v-savrav\\Desktop\\30\\Train_results\\stage3_results.xlsx')

In [54]:
final.columns #Added by Zaheer

Index(['Approval - Act', 'Approval - Pr', 'Paid - Act', 'Paid - Pr', 'Country',
       'Region', 'Frequency', 'CC', 'received_days', 'processing_days',
       'days_to_approval', 'days_to_be_cleared', 'PaidDays', 'NewPaymentTerm',
       'DocumentNo', 'Type', 'Amount in doc. curr.', 'Curr.', 'Vendor',
       'Discount base amount', 'Discount amount', 'Stage'],
      dtype='object')

In [ ]:
#Added by Zaheer
ff=final.rename(columns={'Amount in doc. curr.':'Amount',
             'NewPaymentTerm':'Pay Term'})

In [ ]:
#Added by Zaheer and removed columns (,'Rcvd -Act','Rcvd  -Pr','Procesing - Act','Procesing  - Pr')
final = ff[['DocumentNo',
'Approval - Act','Approval - Pr','Paid - Act','Paid - Pr','CC','Vendor',
'Pay Term','Amount','Curr.','Type','Discount amount','Country','Region','Frequency']]

In [ ]:
#Added by Zaheer
final['stage']='stage3'

In [ ]:
final=final.dropna()

In [ ]:
stage3=final

In [26]:
stage3

,DocumentNo,Approval - Act,Approval - Pr,Paid - Act,Paid - Pr,CC,Vendor,Pay Term,Amount,Curr.,Type,Discount amount,Country,Region,Frequency,stage
2,3700019849,4.0,3.991803,26.0,27.601627,1074.0,3009090,30,-70.7999,AUD,ZJ,0.0,Australia,ASIA,Daily,stage3
7,3700026616,0.0,0.015359,6.0,6.928589,1071.0,3009068,10,-313.0199,GBP,ZJ,0.0,United Kingdom,Europe,Daily,stage3
16,3700019845,1.0,1.001185,26.0,27.667474,1074.0,3009090,30,-77.0000,AUD,ZJ,0.0,Australia,ASIA,Daily,stage3
24,5100125325,1.0,0.999183,52.0,27.508628,1052.0,2264553,30,-105.2400,EUR,ZI,0.0,Netherlands,Europe,Daily,stage3
26,5100125332,1.0,0.999892,52.0,27.318914,1052.0,2264553,30,-1741.2100,EUR,ZI,0.0,Netherlands,Europe,Daily,stage3
30,5100125333,1.0,0.999641,52.0,27.325218,1052.0,2264553,30,-88.9500,EUR,ZI,0.0,Netherlands,Europe,Daily,stage3
31,5100125334,1.0,0.999641,52.0,27.323861,1052.0,2264553,30,-988.4500,EUR,ZI,0.0,Netherlands,Europe,Daily,stage3
35,5100125351,49.0,49.011296,50.0,50.043315,1052.0,2152583,30,-7012.5600,EUR,ZI,0.0,Netherlands,Europe,Daily,stage3
41,3700000830,0.0,0.005074,9.0,7.445340,1589.0,3009068,10,-4978.1200,GBP,ZJ,0.0,United Kingdom,Europe,Daily,stage3
49,5100570472,0.0,0.002527,41.0,28.340928,1062.0,2099531,30,-5437.8000,USD,ZI,0.0,Ireland,Europe,Daily,stage3


In [27]:
import pyodbc 
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=azgovprdsql11;'
                      'Database=PWT;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

In [31]:
for index,row in stage3.iterrows():
    cursor.execute('''INSERT INTO [All_Payment_Term_Train_Stage3_Output]
                   (
                  [DocumentNo],
                  [Approval - Act],[Approval - Pr],
                  [Paid - Act],[Paid - Pr],
                  [CC],[Vendor],[Pay Term],[Amount],[Curr.],[Type],
                  [Discount amount],[Country],[Region],[Frequency],[stage] )
                   
    values (?,?,?,?, ?,?,?,?, ?,?,?,?, ?,?,?,?)''',
    row['DocumentNo'],row['Approval - Act'],row['Approval - Pr'],
                      row['Paid - Act'],row['Paid - Pr'],
                      row['CC'],row['Vendor'],row['Pay Term'],row['Amount'],row['Curr.'],
                      row['Type'],row['Discount amount'],row['Country'],row['Region'],
                      row['Frequency'],row['stage'] )

Error: ('01000', '[01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionRead (recv()). (10051) (SQLExecDirectW); [01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]General network error. Check your network documentation. (11)')

In [ ]:
conn.commit()
conn.close()